<a href="https://colab.research.google.com/github/sanjanb/BiasNet-Pretrained-Model/blob/main/Unsupervised_Pretraining_and_Binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch torchvision matplotlib scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
pip install torchsummary

# **Building the Autoencoder**

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Download MNIST
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Data loaders
train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=64, shuffle=False)


100%|██████████| 9.91M/9.91M [00:00<00:00, 20.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.19MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 5.69MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.09MB/s]


## **Define the Autoencoder Architecture**

In [ ]:
import torch.nn as nn

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28 * 28),
            nn.Tanh(),
            nn.Unflatten(1, (1, 28, 28))
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [ ]:
from torchsummary import summary

model = Autoencoder()
summary(model, input_size=(1, 28, 28))

model = Autoencoder()
print(model)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 128]         100,480
              ReLU-3                  [-1, 128]               0
            Linear-4                   [-1, 64]           8,256
              ReLU-5                   [-1, 64]               0
            Linear-6                  [-1, 128]           8,320
              ReLU-7                  [-1, 128]               0
            Linear-8                  [-1, 784]         101,136
              Tanh-9                  [-1, 784]               0
        Unflatten-10            [-1, 1, 28, 28]               0
Total params: 218,192
Trainable params: 218,192
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.03
Params size (MB): 0.83
Estimated T

## **Pretraining the Autoencoder**

In [ ]:
import torch.optim as optim
from tqdm import tqdm

autoencoder = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
autoencoder.to(device)

# Early stopping logic
best_loss = float('inf')
patience = 5
counter = 0

for epoch in range(50):
    autoencoder.train()
    running_loss = 0.0

    for inputs, _ in tqdm(train_loader):
        inputs = inputs.to(device)
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}')

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        counter = 0
        torch.save(autoencoder.state_dict(), 'pretrained_autoencoder.pth')
        print("Model saved.")
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping.")
            break


100%|██████████| 938/938 [00:20<00:00, 45.97it/s]


Epoch 1, Loss: 0.1091
Model saved.


100%|██████████| 938/938 [00:20<00:00, 45.21it/s]


Epoch 2, Loss: 0.0526
Model saved.


100%|██████████| 938/938 [00:20<00:00, 45.43it/s]


...

Epoch 48, Loss: 0.0224
Model saved.


100%|██████████| 938/938 [00:22<00:00, 42.00it/s]


Epoch 49, Loss: 0.0223
Model saved.


100%|██████████| 938/938 [00:22<00:00, 41.86it/s]

Epoch 50, Loss: 0.0222
Model saved.


# **Fine-Tuning with Binary Classification**

## **Dataset with Even/Odd Labels**

In [ ]:
# Binary labels: 0 for Even, 1 for Odd
def binary_label(label):
    return 0 if label % 2 == 0 else 1

class BinaryMNIST(torch.utils.data.Dataset):
    def __init__(self, original_dataset):
        self.data = original_dataset.data
        self.targets = original_dataset.targets.apply_(binary_label).long()
        original_transforms = original_dataset.transform.transforms
        self.transform = transforms.Compose([t for t in original_transforms if not isinstance(t, transforms.ToTensor)])

    def __getitem__(self, idx):
        img = self.data[idx]
        img = img.float().unsqueeze(0)
        img = self.transform(img)
        label = self.targets[idx]
        return img, label

    def __len__(self):
        return len(self.targets)

binary_train = BinaryMNIST(mnist_train)
binary_test = BinaryMNIST(mnist_test)

train_loader_bin = DataLoader(binary_train, batch_size=64, shuffle=True)
test_loader_bin = DataLoader(binary_test, batch_size=64, shuffle=False)

## **Freeze Encoder and Add Classifier Head**

In [ ]:
class BinaryClassifier(nn.Module):
    def __init__(self, pretrained_encoder):
        super(BinaryClassifier, self).__init__()
        self.encoder = pretrained_encoder
        for param in self.encoder.parameters():
            param.requires_grad = False  # Freeze
        self.classifier = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        features = self.encoder(x)
        out = self.classifier(features)
        return out


## **Training Binary Classifier**

In [ ]:
autoencoder = Autoencoder().to(device)
autoencoder.load_state_dict(torch.load('pretrained_autoencoder.pth'))

model = BinaryClassifier(autoencoder.encoder).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    epoch_loss = 0.0

    for inputs, labels in train_loader_bin:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {epoch_loss / len(train_loader_bin):.4f}")

torch.save(model.state_dict(), "binary_classifier.pth")


Epoch 1, Loss: 5.7800
Epoch 2, Loss: 0.9497
Epoch 3, Loss: 0.2756
Epoch 4, Loss: 0.1725
Epoch 5, Loss: 0.1540
Epoch 6, Loss: 0.1377
Epoch 7, Loss: 0.1237
Epoch 8, Loss: 0.1196
Epoch 9, Loss: 0.1144
Epoch 10, Loss: 0.1085


# **Evaluation + Report**

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader_bin:
        inputs = inputs.to(device)
        outputs = model(inputs)
        preds = (outputs.cpu().numpy() > 0.5).astype(int).flatten()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

print("Accuracy:", accuracy_score(all_labels, all_preds))
print("Precision:", precision_score(all_labels, all_preds))
print("Recall:", recall_score(all_labels, all_preds))
print("Confusion Matrix:\n", confusion_matrix(all_labels, all_preds))


Accuracy: 0.9638
Precision: 0.9725230645808263
Recall: 0.9556562869530942
Confusion Matrix:
 [[4789  137]
 [ 225 4849]]
